# YamboConvergence: automated BSE convergence

This is the highest-level workflow of the plugin. It runs an automated GW or BSE convergence.
Simulations are organized on the fly without human intervention.

The idea is to create a builder instance, which collects all the inputs needed for the simulation, then submit it by means of the aiida "submit" function.

In [79]:
from aiida import orm, load_profile
load_profile()

Profile<uuid='b35700dae723411ea16ebc82d58f16bc' name='mb'>

## 1 - Creation of YamboConvergence instance using protocols

In this tutorial the creation of the builder, for a given workflow, is provided by means of the get_builder_from_protocol function.
This is, for now, just a way to automatically populated all the needed inputs, allowing the user to partially skip the long part of input definition as done in examples like the one contained in ``aiida_yambo/examples_hBN/workflows/yambo_convergence.py``.


It is not meant to be an already tested set of default values as instead is done for the aiida-quantumespresso plugin. 

In [80]:
from aiida.plugins import WorkflowFactory

In [81]:
YamboConvergence = WorkflowFactory('yambo.yambo.yamboconvergence')

## minimal inputs needed for protocols

In [82]:
options = {
    'pwcode_id': 'pw-7.1@hydralogin', 
    'pseudo_family':"PseudoDojo/0.4/PBE/SR/standard/upf",
    'yamboprecode_id':'p2y-5.1@hydralogin',
    'yambocode_id':'yambo-5.1@hydralogin',
    'protocol':'fast',
    #'parent_id':274, #not necessary to set; take your previously nscf id (pk) to skip the DFT part.
    'structure_id':161,
}

In [83]:
#YamboWorkflow.get_builder_from_protocol??

In [84]:
from aiida_quantumespresso.common.types import ElectronicType

In [85]:
builder = YamboConvergence.get_builder_from_protocol(
            pw_code = options['pwcode_id'],
            preprocessing_code = options['yamboprecode_id'],
            code = options['yambocode_id'],
            protocol=options['protocol'],
            protocol_qe=options['protocol'],
            structure=orm.load_node(options['structure_id']),
            overrides={},
            #parent_folder=load_node(options['parent_id']).outputs.remote_folder,
            electronic_type=ElectronicType.INSULATOR, #default is METAL: smearing is used
            calc_type='bse', #or 'bse'; default is 'gw'
)

Summary of the main inputs:
BndsRnXs = 200
NGsBlkXs = 6 Ry
BSENGBlk = 6 Ry
FFTGvecs = 18 Ry


kpoint mesh for nscf: [6, 6, 2]


/scratch/mbonacci/mambaforge/envs/minitutorial/lib/python3.10/site-packages/aiida_quantumespresso/calculations/pw.py:211: UserWarning: `parent_folder` not provided for `nscf` calculation. For work chains wrapping this calculation, you can disable this warning by excluding the `parent_folder` when exposing the inputs of the `PwCalculation`.
  warnings.warn(


In [86]:
#You can also try different protocols:
    
YamboConvergence.get_available_protocols()

{'fast': {'description': 'Fast protocol for a GW convergence: grid -> poor; thresholds -> poor'},
 'moderate': {'description': 'Moderate protocol for a GW convergence: grid -> enough good for standard materials; thresholds -> moderate (5 percent)'},
 'precise': {'description': 'precise protocol for a GW convergence: grid -> same as moderate; thresholds -> precise (1 percent)'},
 'molecule': {'description': 'Moderate protocol for a GW convergence in molecules'}}

Now, if you inspect the prepopulated inputs, you can see the default values respecting the imposed protocol:

In [87]:
builder.ywfl.nscf.pw.parameters.get_dict()

{'CONTROL': {'calculation': 'nscf',
  'forc_conv_thr': 0.001,
  'tprnfor': True,
  'tstress': True,
  'etot_conv_thr': 0.0004},
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'ecutwfc': 60.0,
  'ecutrho': 480.0,
  'force_symmorphic': True,
  'nbnd': 200},
 'ELECTRONS': {'electron_maxstep': 80,
  'mixing_beta': 0.4,
  'conv_thr': 1.6e-09}}

In [88]:
builder.ywfl.yres.yambo.parameters.get_dict()

{'arguments': ['em1s', 'bse', 'bss', 'optics', 'rim_cut', 'dipoles'],
 'variables': {'Chimod': 'hartree',
  'DysSolver': 'n',
  'GTermKind': 'BG',
  'X_and_IO_nCPU_LinAlg_INV': [1, ''],
  'BSEmod': 'resonant',
  'BSKmod': 'SEX',
  'BSSmod': 'd',
  'Lkind': 'full',
  'BSEQptR': [[1, 1], ''],
  'FFTGvecs': [18, 'Ry'],
  'GbndRnge': [[1, 200], ''],
  'BndsRnXs': [[1, 200], ''],
  'NGsBlkXs': [6, 'Ry'],
  'BSENGBlk': [6, 'Ry']}}

## 2 - Inputs completion

We have to include also the resources:

In [89]:
builder.ywfl.scf.pw.metadata.options = {
    'max_wallclock_seconds': 2*60*60, # in seconds
    'resources': {
            "num_machines": 1, # nodes
            "num_mpiprocs_per_machine": 16, # MPI per nodes
            "num_cores_per_mpiproc": 1, # OPENMP
        },
    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed
    #'account':'project_name',
    'queue_name':'s3par',
    #'qos':'',
}

builder.ywfl.nscf.pw.metadata.options = builder.ywfl.scf.pw.metadata.options
builder.ywfl.yres.yambo.metadata.options = builder.ywfl.scf.pw.metadata.options

## 3 - Overrides

It is possible to modify the default inputs also during the builder creation phase, so not a posteriori. This can be done by using overrides:

In [90]:
overrides_scf = {
        'pseudo_family': "PseudoDojo/0.4/PBE/SR/standard/upf", 
        'pw':{
            
        'metadata':{
                    'options':{
                    'max_wallclock_seconds': 60*60, # in seconds
                    'resources': {
                            "num_machines": 1, # nodes
                            "num_mpiprocs_per_machine": 16, # MPI per nodes
                            "num_cores_per_mpiproc": 1, # OPENMP
                        },
                    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed
                    #'account':'project_name',
                    'queue_name':'s3par',
                    #'qos':'',
                                    },
        },
        },
    }

overrides_nscf = {
        'pseudo_family': "PseudoDojo/0.4/PBE/SR/standard/upf", 
        'pw': {
            'parameters':{
                'CONTROL':{}, #not needed if you don't override something
                'SYSTEM':{},
                'ELECTRONS':{'diagonalization':'cg'},
            },
             'metadata':{
                    'options':{
                    'max_wallclock_seconds': 60*60, # in seconds
                    'resources': {
                            "num_machines": 1, # nodes
                            "num_mpiprocs_per_machine": 16, # MPI per nodes
                            "num_cores_per_mpiproc": 1, # OPENMP
                        },
                    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed
                    #'account':'project_name',
                    'queue_name':'s3par',
                    #'qos':'',
                                    },
        },
    },
}

overrides_yambo = {
        "yambo": {
            "parameters": {
                "arguments": [
                    "rim_cut",
                ],
                "variables": {
                    "FFTGvecs": [20, "Ry"],
                    'GbndRnge': [[1, 50], ''],
                    'BndsRnXs': [[1, 50], ''],
                    'NGsBlkXs': [2, 'Ry'],
                    'BSENGBlk': [2, 'Ry'],
                    'KfnQP_E':[[1.5,1,1],''],           # <== Scissor and stretching correction.
                    'BS_CPU':str(int(16/2))+' 2 1',     # <== PARALLELISM INFO
                    'BS_ROLEs':'k eh t',                # <== PARALLELISM INFO
                },
            },
        'metadata':{
                    'options':{
                    'max_wallclock_seconds': 60*60, # in seconds
                    'resources': {
                            "num_machines": 1, # nodes
                            "num_mpiprocs_per_machine": 16, # MPI per nodes
                            "num_cores_per_mpiproc": 1, # OPENMP
                        },
                    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed, i.e. in PBS/Torque 
                    #'account':'project_name',
                    'queue_name':'s3par',
                    #'qos':'',
                                    },
                    },
        },
    
}

#Be careful with the mesh choice!!! 
overrides_meta = {
        'FFTGvecs': {
            'start_ratio': 0.25,
            'stop_ratio': 0.7,
            'delta_ratio': 0.1,
            'max_ratio': 1,
        },
        'bands': {
            'start': 50,
            'stop': 400,
            'delta': 50,
            'max': 600,
            'ratio':[10,25,50],
        },
        'G_vectors': {
            'start': 2,
            'stop': 8,
            'delta': 1,
            'max': 10,
        },
        'kpoint_density': {
            'start': 0.8,
            'stop': 0.2,
            'delta': 3,
            'max': 0.1,
        } ,
        'conv_thr_k': 10,
        'conv_thr_bG': 10,
        'conv_thr_FFT': 10,
        'conv_thr_units': '%', # 'eV'

        
    }

        
overrides_wfl_settings = {
        
        'what':['gap_'],
        'bands_nscf_update':'full-step',
        'skip_pre':False,   
        'type': 'heavy', #or cheap; heavy uses converged value for parameters that we are not converging in a given iteration.
        
    }

overrides = {
        'meta_parameters':overrides_meta,
        'ywfl':{'scf':overrides_scf,'nscf':overrides_nscf,'yres':overrides_yambo},
        'workflow_settings':overrides_wfl_settings,
    }


In [91]:
builder = YamboConvergence.get_builder_from_protocol(
            pw_code = options['pwcode_id'],
            preprocessing_code = options['yamboprecode_id'],
            code = options['yambocode_id'],
            protocol=options['protocol'],
            protocol_qe=options['protocol'],
            structure= orm.load_node(options['structure_id']),
            overrides=overrides,
            #parent_folder=load_node(options['parent_id']).outputs.remote_folder,
            electronic_type=ElectronicType.INSULATOR, #default is METAL: smearing is used
            calc_type='bse', #or 'bse'; default is 'gw'
)

family = orm.load_group("PseudoDojo/0.4/PBE/SR/standard/upf")
#builder.<sublevels_up_to .pw>.pseudos = family.get_pseudos(structure=structure) 
builder.ywfl.scf.pw.pseudos = family.get_pseudos(structure=orm.load_node(161)) 
builder.ywfl.nscf.pw.pseudos = family.get_pseudos(structure=orm.load_node(161)) 

Summary of the main inputs:
BndsRnXs = 50
NGsBlkXs = 2 Ry
BSENGBlk = 2 Ry
FFTGvecs = 20 Ry


kpoint mesh for nscf: [6, 6, 2]


/scratch/mbonacci/mambaforge/envs/minitutorial/lib/python3.10/site-packages/aiida_quantumespresso/calculations/pw.py:211: UserWarning: `parent_folder` not provided for `nscf` calculation. For work chains wrapping this calculation, you can disable this warning by excluding the `parent_folder` when exposing the inputs of the `PwCalculation`.
  warnings.warn(


## spiega che fai il gruppo cos' hai tutto li, tipo caching ma in realtà non duplichi... per yambo é importante:
- non hai problemi con cleaned remote
- dati pesanti tipo ndb.QP

In [92]:
try:
    g = orm.load_group('tutorial/hBN/convergence')
except:
    g = orm.Group('tutorial/hBN/convergence')
    g.store()

In [93]:
builder.group_label = orm.Str('tutorial/hBN/convergence') # verdi group create tutorial/hBN/convergence; all calculationsc are added to the group

In [94]:
builder.ywfl.yres.yambo.parameters = orm.Dict(
    dict={'arguments':['em1s','bse','bss','optics', 'dipoles',],
                'variables':{
                'BSEmod': 'resonant',
                'BSKmod': 'SEX',
                'BSSmod': 'd',
                'Lkind': 'full',
                'NGsBlkXs': [2, 'Ry'],
                'BSENGBlk': [2, 'Ry'],
                'Chimod': 'hartree',
                'DysSolver': 'n',
                'BEnSteps': [10,''],
                'BSEQptR': [[1,1],''],
                'BSEBands': [[8,9],''],
                'BEnRange': [[0.0, 10.0],'eV'],
                'BDmRange': [[0.1, 0.1],'eV'],
                'BLongDir': [[1.0, 1.0, 1.0],''],
                'LongDrXp': [[1.0, 1.0, 1.0],''],
                'LongDrXd': [[1.0, 1.0, 1.0],''],
                'LongDrXs': [[1.0, 1.0, 1.0],''],
                'BndsRnXs': [[1,50], ''],
                'KfnQP_E':[[1.5,1,1],''],
                'BS_CPU':str(int(16/2))+' 2 1',
                'BS_ROLEs':'k eh t',
                },}
)

In [95]:
builder.parameters_space.get_list()

[{'var': ['FFTGvecs'],
  'start': 21,
  'stop': 58,
  'delta': 8,
  'max': 84,
  'steps': 4,
  'max_iterations': 4,
  'conv_thr': 10,
  'conv_thr_units': '%',
  'convergence_algorithm': 'new_algorithm_1D'},
 {'var': ['kpoint_mesh'],
  'start': [4, 4, 2],
  'stop': [16, 16, 6],
  'delta': [3, 3, 3],
  'max': [30, 30, 10],
  'steps': 4,
  'max_iterations': 4,
  'conv_thr': 10,
  'conv_thr_units': '%',
  'convergence_algorithm': 'new_algorithm_1D'},
 {'var': ['BndsRnXp', 'GbndRnge', 'NGsBlkXp'],
  'start': [80, 80, 2],
  'stop': [400, 400, 8],
  'delta': [50, 50, 1],
  'max': [600, 600, 10],
  'steps': 6,
  'max_iterations': 8,
  'conv_thr': 10,
  'conv_thr_units': '%',
  'convergence_algorithm': 'new_algorithm_2D'}]

In [96]:
builder.parameters_space = orm.List(builder.parameters_space.get_list()[-2:-1])

In [97]:
builder.parameters_space.get_list()


[{'var': ['kpoint_mesh'],
  'start': [4, 4, 2],
  'stop': [16, 16, 6],
  'delta': [3, 3, 3],
  'max': [30, 30, 10],
  'steps': 4,
  'max_iterations': 4,
  'conv_thr': 10,
  'conv_thr_units': '%',
  'convergence_algorithm': 'new_algorithm_1D'}]

## 3 RUN

In [98]:
from aiida.engine import submit

In [99]:
run = None

In [100]:
if run:
    print('run is already running -> {}'.format(run.pk))
    print('sure that you want to run again?, if so, copy the else instruction in the cell below and run!')
else:
    run = submit(builder)

print(run)

/scratch/mbonacci/mambaforge/envs/minitutorial/lib/python3.10/site-packages/aiida_quantumespresso/calculations/pw.py:211: UserWarning: `parent_folder` not provided for `nscf` calculation. For work chains wrapping this calculation, you can disable this warning by excluding the `parent_folder` when exposing the inputs of the `PwCalculation`.
  warnings.warn(


uuid: 6457a176-4933-41ae-aeaf-1a9378ebc386 (pk: 6185) (aiida.workflows:yambo.yambo.yamboconvergence)


## Inspecting the workflow

report, e mostra che fa prima le bande...

# Inspecting the outputs

suppose that your calculation completed successfully, then you can access the outputs via the output method of the run instance: 

In [101]:
run.is_finished_ok

False

In [102]:
run.outputs. #+TAB

SyntaxError: invalid syntax (708973465.py, line 1)

The converged parameters can be obtained via the "infos" output Dict:

In [103]:
run.outputs.infos.get_dict()

NotExistentAttributeError: Node<6185> does not have an output with link label 'infos'

The full convergence history can be visualized in a table form using pandas:

In [26]:
import pandas as pd

In [27]:
history = run.outputs.history.get_dict()

In [28]:
history_table = pd.DataFrame(history)

In [29]:
history_table

gap_                                  uuid  failed  useful  BndsRnXp  \
0   1.106919  7f687803-8958-4807-857a-eabc39a2f27f   False   False       150   
1   1.109694  084b6d90-7aa3-4a4b-942f-23219693cddf   False   False       150   
2   1.109486  e61184fe-fa0c-417d-aeba-8cc8156e8081   False   False       150   
3   1.109428  84443b47-b5a3-471e-8fc7-8a100ad399b7   False   False       150   
4   1.106919  7f687803-8958-4807-857a-eabc39a2f27f   False   False       150   
5   0.960021  ce33e842-c988-4640-b20a-e86dacfb25ef   False   False       150   
6   0.971251  884078cf-9d80-4301-8902-0f62ad5b9091   False   False       150   
7   0.963438  cf9d793a-eee3-4253-b486-799fdcf175e2   False   False       150   
8   0.835478  a415b238-056d-4b07-9d24-8dae00f09510   False   False        60   
9   0.841587  31ad3b67-ef3e-43ce-9a2a-e03582d62406   False   False       400   
10  1.071505  73b873cf-1557-49bd-8e25-5a1198f2517b   False   False        60   
11  1.031643  b0c8a9f3-be77-427c-974e-fc7764ddc0e3   False   False       160   
12  0.960864  def81756-0450-4518-bdb4-d6286a9bf664   False   False       310   
13  1.115209  3321f163-239f-437f-9d1f-65cafddb05f5   False   False       400   
14  0.840785  b87eecec-2c90-4cc2-b13c-705689923586   False   False       110   
15  0.841551  ad19c1a6-fcc1-4eab-a1ad-de7dad716bf7   False   False       450   
16  1.105343  50362d13-9635-4003-83bf-08c4ce4ba56a   False   False       110   
17  1.066398  c191fd4a-d499-409b-8ed0-f7927e337ef3   False   False       210   
18  1.032749  49f692e7-00da-48ce-b006-bfd6055ea813   False   False       360   
19  1.115211  cc317b15-b7ba-48b1-9669-b55024e7ce71   False   False       450   
20  1.026851  24a4bd74-54f5-4758-b61c-edf3da3e5c91   False    True       110   

    FFTGvecs  GbndRnge  NGsBlkXp  global_step   kpoint_mesh  \
0         12       150       4.0            1     [4, 4, 4]   
1         22       150       4.0            2     [4, 4, 4]   
2         27       150       4.0            3     [4, 4, 4]   
3         35       150       4.0            4     [4, 4, 4]   
4         12       150       4.0            5     [4, 4, 4]   
5         12       150       4.0            6     [7, 7, 7]   
6         12       150       4.0            7  [10, 10, 10]   
7         12       150       4.0            8  [12, 12, 12]   
8         12        60       2.0            9     [7, 7, 7]   
9         12       400       2.0           10     [7, 7, 7]   
10        12        60       8.0           11     [7, 7, 7]   
11        12       160       6.0           12     [7, 7, 7]   
12        12       310       4.0           13     [7, 7, 7]   
13        12       400       8.0           14     [7, 7, 7]   
14        12       110       3.0           15     [7, 7, 7]   
15        12       450       3.0           16     [7, 7, 7]   
16        12       110       9.0           17     [7, 7, 7]   
17        12       210       7.0           18     [7, 7, 7]   
18        12       360       5.0           19     [7, 7, 7]   
19        12       450       9.0           20     [7, 7, 7]   
20        12       110       5.0           21     [7, 7, 7]   

              parameters_studied  
0                       FFTGvecs  
1                       FFTGvecs  
2                       FFTGvecs  
3                       FFTGvecs  
4                    kpoint_mesh  
5                    kpoint_mesh  
6                    kpoint_mesh  
7                    kpoint_mesh  
8   BndsRnXp, GbndRnge, NGsBlkXp  
9   BndsRnXp, GbndRnge, NGsBlkXp  
10  BndsRnXp, GbndRnge, NGsBlkXp  
11  BndsRnXp, GbndRnge, NGsBlkXp  
12  BndsRnXp, GbndRnge, NGsBlkXp  
13  BndsRnXp, GbndRnge, NGsBlkXp  
14  BndsRnXp, GbndRnge, NGsBlkXp  
15  BndsRnXp, GbndRnge, NGsBlkXp  
16  BndsRnXp, GbndRnge, NGsBlkXp  
17  BndsRnXp, GbndRnge, NGsBlkXp  
18  BndsRnXp, GbndRnge, NGsBlkXp  
19  BndsRnXp, GbndRnge, NGsBlkXp  
20  BndsRnXp, GbndRnge, NGsBlkXp

The last calculations can be obtained using:

In [30]:
history_table[history_table['useful']==True]

gap_                                  uuid  failed  useful  BndsRnXp  \
20  1.026851  24a4bd74-54f5-4758-b61c-edf3da3e5c91   False    True       110   

    FFTGvecs  GbndRnge  NGsBlkXp  global_step kpoint_mesh  \
20        12       110       5.0           21   [7, 7, 7]   

              parameters_studied  
20  BndsRnXp, GbndRnge, NGsBlkXp

In [28]:
# some fine plotting of the results, similar to the ones in the paper.